# ⚡ Sofascore Multi-League Scraper v4 (PARALLEL)

**Fast parallel scraper with:**
- 🚀 Multi-threaded API calls (5-10x faster)
- 🌍 10+ major world leagues
- 📅 3 years of historical data
- 💰 Full odds (1X2, BTTS, O/U)
- 📊 Team streaks & H2H

In [1]:
!pip install tls_client pandas numpy -q
print("Dependencies installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 15.8 MB/s eta 0:00:00
Dependencies installed!


In [2]:
#@title Configuration { run: "auto" }
#@markdown ### Leagues:
Premier_League = True #@param {type:"boolean"}
La_Liga = True #@param {type:"boolean"}
Bundesliga = True #@param {type:"boolean"}
Serie_A = True #@param {type:"boolean"}
Ligue_1 = True #@param {type:"boolean"}
Liga_MX = True #@param {type:"boolean"}
Eredivisie = False #@param {type:"boolean"}
Primeira_Liga = False #@param {type:"boolean"}
MLS = False #@param {type:"boolean"}
Brazilian_Serie_A = False #@param {type:"boolean"}

#@markdown ### Options:
Years_of_Data = 3 #@param {type:"slider", min:1, max:5, step:1}
Max_Matches_Per_Team = 30 #@param {type:"slider", min:10, max:50, step:5}
Parallel_Workers = 10 #@param {type:"slider", min:2, max:10, step:1}
Include_Odds = True #@param {type:"boolean"}
Include_Streaks = True #@param {type:"boolean"}
Include_H2H = True #@param {type:"boolean"}

LEAGUES = {
    'Premier League': {'id': 17, 'enabled': Premier_League},
    'La Liga': {'id': 8, 'enabled': La_Liga},
    'Bundesliga': {'id': 35, 'enabled': Bundesliga},
    'Serie A': {'id': 23, 'enabled': Serie_A},
    'Ligue 1': {'id': 34, 'enabled': Ligue_1},
    'Liga MX': {'id': 11621, 'enabled': Liga_MX},
    'Eredivisie': {'id': 37, 'enabled': Eredivisie},
    'Primeira Liga': {'id': 238, 'enabled': Primeira_Liga},
    'MLS': {'id': 242, 'enabled': MLS},
    'Brazilian Serie A': {'id': 325, 'enabled': Brazilian_Serie_A},
}

selected = {k: v for k, v in LEAGUES.items() if v['enabled']}
print(f"Selected: {len(selected)} leagues | {Years_of_Data} years | {Parallel_Workers} workers")

Selected: 6 leagues | 3 years | 10 workers


In [3]:
import time
import random
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from tls_client import Session
import pandas as pd
import numpy as np
from datetime import datetime

# Thread-safe session pool
class SessionPool:
    def __init__(self, size=5):
        self.sessions = [Session(client_identifier="firefox_120") for _ in range(size)]
        self.index = 0
        self.lock = Lock()

    def get(self):
        with self.lock:
            session = self.sessions[self.index % len(self.sessions)]
            self.index += 1
            return session

pool = SessionPool(Parallel_Workers)
BASE_URL = "https://www.sofascore.com/api/v1"
request_count = 0
count_lock = Lock()

def fetch_json(url, retries=2):
    global request_count
    full_url = f"{BASE_URL}{url}" if url.startswith('/') else url
    session = pool.get()

    for attempt in range(retries + 1):
        try:
            time.sleep(random.uniform(0.2, 0.5))  # Reduced delay for parallel
            response = session.get(full_url)

            with count_lock:
                request_count += 1

            if response.status_code == 200:
                return response.json()
            elif response.status_code == 403:
                time.sleep(3)  # Back off on 403
        except:
            if attempt < retries:
                time.sleep(1)
    return None

def convert_fractional(frac_str):
    try:
        if '/' in str(frac_str):
            num, den = map(int, str(frac_str).split('/'))
            return round(1 + (num / den), 3)
        return float(frac_str)
    except:
        return None

def slugify(text):
    return re.sub(r'[^a-z0-9]+', '_', text.lower()).strip('_')

print(f"Session pool ready with {Parallel_Workers} workers")

Session pool ready with 10 workers


In [4]:
# Data fetching functions (optimized for parallel use)

def get_seasons(tournament_id, num_years=3):
    data = fetch_json(f"/unique-tournament/{tournament_id}/seasons")
    if not data or 'seasons' not in data:
        return []
    return [{'id': s['id'], 'name': s['name']} for s in data['seasons'][:num_years * 2]]

def get_teams(tournament_id, season_id):
    data = fetch_json(f"/unique-tournament/{tournament_id}/season/{season_id}/standings/total")
    if not data or 'standings' not in data:
        return []
    teams = []
    for s in data['standings']:
        for r in s.get('rows', []):
            t = r.get('team', {})
            teams.append({'id': t.get('id'), 'name': t.get('name')})
    return teams

def get_team_matches(team_id, tournament_id, max_pages=3):
    matches = []
    for page in range(max_pages):
        data = fetch_json(f"/team/{team_id}/events/last/{page}")
        if not data or 'events' not in data:
            break
        for e in data.get('events', []):
            if e.get('status', {}).get('type') != 'finished':
                continue
            if e.get('tournament', {}).get('uniqueTournament', {}).get('id') != tournament_id:
                continue
            matches.append({
                'match_id': e.get('id'),
                'date': datetime.fromtimestamp(e.get('startTimestamp', 0)).strftime('%Y-%m-%d'),
                'timestamp': e.get('startTimestamp', 0),
                'home_team': e.get('homeTeam', {}).get('name'),
                'home_team_id': e.get('homeTeam', {}).get('id'),
                'away_team': e.get('awayTeam', {}).get('name'),
                'away_team_id': e.get('awayTeam', {}).get('id'),
                'home_score': e.get('homeScore', {}).get('current'),
                'away_score': e.get('awayScore', {}).get('current'),
                'tournament_id': tournament_id,
            })
    return matches

def enrich_match(match, include_odds, include_streaks, include_h2h):
    """Enrich a single match with stats, odds, streaks, h2h."""
    match_id = match['match_id']

    # Stats
    stats_data = fetch_json(f"/event/{match_id}/statistics")
    if stats_data and 'statistics' in stats_data:
        for period in stats_data.get('statistics', []):
            pname = period.get('period', 'ALL').lower()
            for g in period.get('groups', []):
                for item in g.get('statisticsItems', []):
                    name = item.get('name', '').lower().replace(' ', '_')
                    key = name if pname == 'all' else f"{pname}_{name}"
                    match[f"{key}_home"] = item.get('home')
                    match[f"{key}_away"] = item.get('away')

    # Odds
    if include_odds:
        odds_data = fetch_json(f"/event/{match_id}/odds/1/all")
        if odds_data and 'markets' in odds_data:
            for market in odds_data.get('markets', []):
                mid = market.get('marketId')
                for choice in market.get('choices', []):
                    name = choice.get('name', '')
                    dec = convert_fractional(choice.get('fractionalValue', ''))
                    if not dec: continue
                    if mid == 1:
                        if name == '1': match['odds_1x2_home'] = dec
                        elif name == 'X': match['odds_1x2_draw'] = dec
                        elif name == '2': match['odds_1x2_away'] = dec
                    elif mid == 5:
                        if name.lower() == 'yes': match['odds_btts_yes'] = dec
                        elif name.lower() == 'no': match['odds_btts_no'] = dec

    # Streaks
    if include_streaks:
        streak_data = fetch_json(f"/event/{match_id}/team-streaks")
        if streak_data:
            for item in streak_data.get('general', []):
                name = item.get('name', '')
                team = item.get('team', '')
                val = item.get('value', '')
                if '/' in str(val):
                    parts = str(val).split('/')
                    match[f"streak_{team}_{slugify(name)}"] = int(parts[0])

    # H2H (leakage-free)
    if include_h2h:
        h2h_data = fetch_json(f"/event/{match_id}/h2h/events")
        if h2h_data and 'events' in h2h_data:
            h2h_home = h2h_away = h2h_draws = 0
            for h2h in h2h_data.get('events', []):
                if h2h.get('startTimestamp', 0) >= match['timestamp']:
                    continue
                if h2h.get('status', {}).get('type') != 'finished':
                    continue
                winner = h2h.get('winnerCode')
                h2h_home_id = h2h.get('homeTeam', {}).get('id')
                if winner == 1:
                    if h2h_home_id == match['home_team_id']: h2h_home += 1
                    else: h2h_away += 1
                elif winner == 2:
                    if h2h_home_id == match['home_team_id']: h2h_away += 1
                    else: h2h_home += 1
                elif winner == 3:
                    h2h_draws += 1
            match['h2h_home_wins'] = h2h_home
            match['h2h_away_wins'] = h2h_away
            match['h2h_draws'] = h2h_draws

    return match

print("Data functions ready")

Data functions ready


In [5]:
def scrape_league_parallel(league_name, tournament_id, num_years, max_matches, workers,
                           include_odds, include_streaks, include_h2h):
    """Scrape a single league using parallel workers."""
    print(f"\n{'='*50}")
    print(f"SCRAPING: {league_name}")
    print(f"{'='*50}")

    # Get seasons and teams
    seasons = get_seasons(tournament_id, num_years)
    if not seasons:
        print(f"No seasons found")
        return pd.DataFrame()

    all_teams = {}
    for s in seasons[:num_years]:
        for t in get_teams(tournament_id, s['id']):
            if t['id']: all_teams[t['id']] = t['name']

    print(f"Found {len(seasons)} seasons, {len(all_teams)} teams")

    # Collect all matches (parallel by team)
    all_matches = []
    team_ids = list(all_teams.keys())

    print(f"Collecting matches from {len(team_ids)} teams...")
    with ThreadPoolExecutor(max_workers=workers) as executor:
        futures = {executor.submit(get_team_matches, tid, tournament_id, 3): tid for tid in team_ids}
        done = 0
        for future in as_completed(futures):
            matches = future.result()
            all_matches.extend(matches[:max_matches])
            done += 1
            print(f"\r  Teams: {done}/{len(team_ids)} | Matches: {len(all_matches)}", end='', flush=True)

    # Deduplicate
    seen = set()
    unique_matches = []
    for m in all_matches:
        if m['match_id'] not in seen:
            seen.add(m['match_id'])
            m['league'] = league_name
            unique_matches.append(m)

    print(f"\n  Unique matches: {len(unique_matches)}")

    # Enrich matches in parallel
    print(f"Enriching with stats/odds/h2h (parallel)...")
    enriched = []
    with ThreadPoolExecutor(max_workers=workers) as executor:
        futures = [executor.submit(enrich_match, m, include_odds, include_streaks, include_h2h)
                   for m in unique_matches]
        done = 0
        for future in as_completed(futures):
            enriched.append(future.result())
            done += 1
            if done % 10 == 0:
                print(f"\r  Enriched: {done}/{len(unique_matches)}", end='', flush=True)

    print(f"\n  Completed: {len(enriched)} matches")
    return pd.DataFrame(enriched)

print("Parallel scraper ready")

Parallel scraper ready


In [ ]:
#@title Run Scraping { display-mode: "form" }

all_data = []
start_time = time.time()

print("="*60)
print("MULTI-LEAGUE PARALLEL SCRAPER")
print("="*60)
print(f"Leagues: {len(selected)}")
print(f"Workers: {Parallel_Workers}")
print(f"Years: {Years_of_Data}")

for name, info in selected.items():
    try:
        df = scrape_league_parallel(
            league_name=name,
            tournament_id=info['id'],
            num_years=Years_of_Data,
            max_matches=Max_Matches_Per_Team,
            workers=Parallel_Workers,
            include_odds=Include_Odds,
            include_streaks=Include_Streaks,
            include_h2h=Include_H2H
        )
        if len(df) > 0:
            all_data.append(df)
    except Exception as e:
        print(f"\nError in {name}: {e}")

# Combine
if all_data:
    df_final = pd.concat(all_data, ignore_index=True)
    df_final = df_final.drop_duplicates(subset=['match_id'], keep='first')

    elapsed = time.time() - start_time
    print("\n" + "="*60)
    print("COMPLETE!")
    print("="*60)
    print(f"Total matches: {len(df_final)}")
    print(f"Columns: {len(df_final.columns)}")
    print(f"Time: {elapsed/60:.1f} minutes")
    print(f"API requests: {request_count}")
    print(f"Leagues: {df_final['league'].nunique()}")

    if 'odds_1x2_home' in df_final.columns:
        print(f"Odds coverage: {df_final['odds_1x2_home'].notna().mean()*100:.1f}%")
else:
    df_final = pd.DataFrame()
    print("No data collected")

MULTI-LEAGUE PARALLEL SCRAPER
Leagues: 6
Workers: 10
Years: 3

SCRAPING: Premier League
Found 6 seasons, 25 teams
  Teams: 25/25 | Matches: 678
  Unique matches: 419
Enriching with stats/odds/h2h (parallel)...
  Enriched: 410/419
  Completed: 419 matches

SCRAPING: La Liga
Found 6 seasons, 26 teams
  Teams: 26/26 | Matches: 697
  Unique matches: 440
Enriching with stats/odds/h2h (parallel)...
  Enriched: 440/440
  Completed: 440 matches

SCRAPING: Bundesliga
Found 6 seasons, 21 teams
  Teams: 21/21 | Matches: 611
  Unique matches: 374
Enriching with stats/odds/h2h (parallel)...
  Enriched: 50/374

In [ ]:
# Preview
if len(df_final) > 0:
    print("Sample:")
    display(df_final[['date', 'league', 'home_team', 'away_team', 'home_score', 'away_score']].head(10))

    print("\nBy League:")
    print(df_final['league'].value_counts())

In [ ]:
from google.colab import files

if len(df_final) > 0:
    filename = f"sofascore_parallel_{len(selected)}lg_{Years_of_Data}yr_{len(df_final)}matches.csv"
    df_final.to_csv(filename, index=False)
    print(f"Saved: {filename}")
    files.download(filename)

---
## Speed Comparison

| Workers | Estimated Speed |
|---------|----------------|
| 1 (serial) | 1x baseline |
| 3 workers | ~2.5x faster |
| 5 workers | ~4x faster |
| 10 workers | ~6x faster (may hit rate limits) |

### Tips:
- Start with 5 workers
- If you get 403 errors, reduce workers
- Each league takes ~2-5 minutes with 5 workers